In [18]:
import copy, sys
import random

sys.path.append("../../../")
from importlib import reload

import utils.display.tables as tables
import inferring.InferringDFA as InferringDFA
import utils.runLearnLib.RunLearnLib as RunLearnLib
import utils.automata.DFA
import utils.advice_systems.SRS as SRS

reload(InferringDFA)
reload(RunLearnLib)
reload(SRS)
reload(utils.automata.DFA)
reload(tables)

from utils.runLearnLib.RunLearnLib import RunLearnLib
from inferring.InferringDFA import InferringDFA
from utils.automata.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS
from utils.display.tables import Tables

In [19]:
class Record:

    def __init__(self, dfa_sz, cnt_normal, cnt_with_advice):
        self.dfa_sz = dfa_sz
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice

        self.mq_reduction = int(
            ((cnt_normal[0] - cnt_with_advice[0]) / cnt_normal[0]) * 100)
        self.eq_reduction = int(
            ((cnt_normal[1] - cnt_with_advice[1]) / cnt_normal[1]) * 100)

    def print_record(self):
        print(
            f"|Q| = {self.dfa_sz}, |cnt_normal| = {self.cnt_normal} vs |cnt_with_partial_advice| = {self.cnt_with_partial_advice}"
        )

## Partial Specification 
In this experiment we learn language defined by random DFA. As an advice system, we use partial DFA, which is constructed by pruning target DFA. <br />
The degree of pruning is specified by an argument passed to the function **prune()** according to the specification of this function. 

In [23]:
number_of_itreations = 100
#  Fixed random seeds for reproducibility
seeds = [i+1 for i in range(number_of_itreations)]
# The alphabet of the DFA
input_signs = ['a', 'b', 'c', 'd']
# The bound on the number of states
max_number_of_states = 1000
# The list to aggregate the results
results = []

for i, seed in enumerate(seeds):
    print(f"Iteration {i}")
    random.seed(seed)

    # Create a random DFA
    dfa = DFA()
    number_of_states = random.randint(max_number_of_states // 2,
                                      max_number_of_states)
    dfa.create_random_dfa(Q=number_of_states, input_signs=input_signs)

    # Create a partial DFA based on dfa, the advice system will be created based on dfa_partial 
    dfa_partial = copy.deepcopy(dfa)
    dfa_partial.prune()
    # To ensure rewriting words from the beginning of word
    dfa_to_learn = dfa.create_copy_with_start_sign()

    # Run learning process with an oracle
    learn_dfa_with_advice = InferringDFA(copy.deepcopy(dfa_to_learn),
                                         advice_system=dfa_partial,
                                         check_consistency=True)
    minimal_dfa, learning_info_with_advice = learn_dfa_with_advice.run()

    # Run learning of the language of DFA
    learn_dfa = InferringDFA(copy.deepcopy(minimal_dfa))
    _, learning_info = learn_dfa.run()

    results.append(
        Record(dfa_sz=minimal_dfa.Q,
               cnt_normal=learning_info,
               cnt_with_advice=learning_info_with_advice))

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [24]:
results.sort(key = lambda x: x.dfa_sz)

In [25]:
columns = [('', 'DFA'), ('L* without advice', 'MQ'), ('L* without advice', 'EQ'), ('L* with advice', 'MQ'), ('L* with advice', 'EQ'), ('Reduction', 'MQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.dfa_sz for r in results],
    columns[1]: [r.cnt_normal[0] for r in results],
    columns[2]: [r.cnt_normal[1] for r in results],
    columns[3]: [r.cnt_with_advice[0] for r in results],
    columns[4]: [r.cnt_with_advice[1] for r in results],
    columns[5]: [r.mq_reduction for r in results],
    columns[6]: [r.eq_reduction for r in results]
}

table_creator = Tables()
df = table_creator.create_basic_table(columns=columns, data=data, reduction_columns=[('Reduction', 'MQ'), ("Reduction", "EQ")])
display(df)

L* without advice     L* with advice     Reduction    
                   DFA                MQ  EQ             MQ  EQ        MQ  EQ
0                  502            117024  40          84952  39       27%  2%
1                  510            110724  42          80176  41       27%  2%
2                  514            128048  47          93156  46       27%  2%
3                  527             99650  39          71764  38       27%  2%
4                  527            118631  42          86011  41       27%  2%
..                 ...               ...  ..            ...  ..       ...  ..
99                 983            252695  45         183947  44       27%  2%
100                                                                          
101      Max reduction                                                28%  2%
102      Min reduction                                                25%  0%
103  Average reduction                                                26%  1%

[104 rows x 7 columns]

In [26]:
import pandas as pd 
with pd.option_context('display.max_colwidth', None, 'display.max_rows', None): 
    display(df)

L* without advice     L* with advice     Reduction    
                   DFA                MQ  EQ             MQ  EQ        MQ  EQ
0                  502            117024  40          84952  39       27%  2%
1                  510            110724  42          80176  41       27%  2%
2                  514            128048  47          93156  46       27%  2%
3                  527             99650  39          71764  38       27%  2%
4                  527            118631  42          86011  41       27%  2%
5                  528            127308  44          92518  43       27%  2%
6                  531            142906  54         104208  53       27%  1%
7                  532            121885  35          88424  34       27%  2%
8                  532            124014  43          90022  42       27%  2%
9                  533            137045  49          99797  48       27%  2%
10                 536            150686  52         110018  51       26%  1%
11                 543             98328  36          70678  35       28%  2%
12                 548            125549  40          91080  39       27%  2%
13                 557            103091  39          74171  38       28%  2%
14                 557            118694  43          85882  42       27%  2%
15                 559            123594  40          89548  39       27%  2%
16                 559            157149  57         114733  56       26%  1%
17                 565            129442  45          93902  44       27%  2%
18                 572            108155  37          77884  36       27%  2%
19                 585            126999  42          91951  41       27%  2%
20                 591            128301  44          92893  43       27%  2%
21                 592            142732  38         103718  37       27%  2%
22                 599            158801  39         115737  38       27%  2%
23                 600            161467  52         117732  51       27%  1%
24                 600            130254  36          94306  35       27%  2%
25                 601            147306  42         107098  41       27%  2%
26                 610            142188  42         103204  41       27%  2%
27                 610            164157  49         119692  48       27%  2%
28                 615            145814  47         107742  47       26%  0%
29                 616            148516  42         109767  42       26%  0%
30                 630            151890  46         110368  45       27%  2%
31                 633            119684  32          88080  32       26%  0%
32                 633            155146  47         112794  46       27%  2%
33                 645            142600  40         105244  40       26%  0%
34                 654            149823  47         108676  46       27%  2%
35                 655            173641  47         126545  46       27%  2%
36                 655            150052  45         110808  45       26%  0%
37                 668            166394  43         121030  42       27%  2%
38                 671            161771  36         117543  35       27%  2%
39                 677            157799  45         114527  44       27%  2%
40                 678            152606  44         110624  43       27%  2%
41                 678            196014  53         145235  53       25%  0%
42                 682            164422  45         119468  44       27%  2%
43                 684            184063  43         134196  42       27%  2%
44                 692            166832  42         121218  41       27%  2%
45                 693            147662  40         106826  39       27%  2%
46                 699            176910  49         128740  48       27%  2%
47                 699            202083  50         147631  49       26%  2%
48                 700            168760  47         124719  47       26%  0%
49                 714            232131  56 